### Importação Bibliotecas

---

In [272]:
import mlflow
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn import tree
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
import time


### Constantes

---

In [273]:
ROOT_PATH = '../../data/'
SEED = 42
TARGET_COL = "default.payment.next.month"


Definir a diretoria onde as experiências são guardadas

In [274]:
from pathlib import Path

#uri = "mlruns"
uri = "http://localhost:5000"
#Path(uri).mkdir(parents=True, exist_ok=True)

mlflow.set_tracking_uri(uri)

In [275]:
mlflow.set_experiment("Rumos Bank Lending")

<Experiment: artifact_location='mlflow-artifacts:/395285683720049485', creation_time=1744151061274, experiment_id='395285683720049485', last_update_time=1744151061274, lifecycle_stage='active', name='Rumos Bank Lending', tags={}>

Dataset

In [276]:
df = pd.read_csv(ROOT_PATH + 'lending_data.csv')
df = df.sample(frac=0.01, random_state=SEED)

In [277]:
train_set, test_set = train_test_split(df, test_size = 0.2, random_state = SEED)

In [278]:
X_train = train_set.drop(['default.payment.next.month'], axis = 'columns')
y_train = train_set['default.payment.next.month']

X_test = test_set.drop(['default.payment.next.month'], axis = 1)
y_test = test_set['default.payment.next.month']

### Guardar datasets, modelos, artefactos, métricas e parametros da run

In [280]:
# guardarmos o dataset de treino e de teste associado à run
train_dataset = mlflow.data.from_pandas(train_set, source="split_from_full_dataset", targets=TARGET_COL, name="Rumos Bank Lending Dataset")
test_dataset = mlflow.data.from_pandas(test_set, source="split_from_full_dataset", targets=TARGET_COL, name="Rumos Bank Lending Dataset")

c:\Users\PereiraRodrigues\miniconda3\envs\rumos_bank_lending\Lib\site-packages\mlflow\data\dataset_source_registry.py:149: UserWarning: Failed to determine whether UCVolumeDatasetSource can resolve source information for 'split_from_full_dataset'. Exception: 
  return _dataset_source_registry.resolve(
c:\Users\PereiraRodrigues\miniconda3\envs\rumos_bank_lending\Lib\site-packages\mlflow\data\dataset_source_registry.py:149: UserWarning: The specified dataset source can be interpreted in multiple ways: LocalArtifactDatasetSource, LocalArtifactDatasetSource. MLflow will assume that this is a LocalArtifactDatasetSource source.
  return _dataset_source_registry.resolve(
c:\Users\PereiraRodrigues\miniconda3\envs\rumos_bank_lending\Lib\site-packages\mlflow\data\dataset_source_registry.py:149: UserWarning: Failed to determine whether UCVolumeDatasetSource can resolve source information for 'split_from_full_dataset'. Exception: 
  return _dataset_source_registry.resolve(
c:\Users\PereiraRodrigue

In [ ]:
models_and_params = [

    ("logistic_regression",
     Pipeline([
         ("Classifier", LogisticRegression(max_iter=500, solver='lbfgs', random_state=SEED, class_weight='balanced'))
     ]),
     {
         "Classifier__C": [0.001, 0.01, 0.1, 1, 10, 100]
     }),

    ("knn",
     Pipeline([
         ("Classifier", KNeighborsClassifier())
     ]),
     {
         "Classifier__n_neighbors": list(range(1, 10))
     }),

         ("svc",
     Pipeline([
         ("Classifier", SVC(random_state=SEED, class_weight='balanced', gamma='scale', probability=True, verbose=True))
     ]),
     {
         "Classifier__C": [0.1, 1, 10],
         "Classifier__kernel": ['rbf', 'linear']
     }),

    ("decision_tree",
     Pipeline([
         ("Classifier", tree.DecisionTreeClassifier(random_state=SEED, class_weight='balanced'))
     ]),
     {
         "Classifier__max_depth": [3, 6],
         "Classifier__min_samples_split": [2, 4, 10]
     }),

    ("random_forest",
     Pipeline([
         ("Classifier", RandomForestClassifier(random_state=SEED, class_weight='balanced'))
     ]),
     {
         "Classifier__n_estimators": [10, 100, 300, 1000]
     }),

    ("mlp",
     Pipeline([
         ("Classifier", MLPClassifier(solver='lbfgs', random_state=SEED, max_iter=1000))
     ]),
     {
         "Classifier__hidden_layer_sizes": [(20,), (20, 10), (20, 10, 2)],
         "Classifier__learning_rate_init": [0.0001, 0.001, 0.01, 0.1]
     })
]

In [288]:
for model_name, pipeline, param_grid in models_and_params:

    # Iniciar Uma run no MlFlow
    run = mlflow.start_run(run_name="Ml Model Run - pipeline")
    RUN_ID = run.info.run_uuid


    # Guardar Informação Dataset de treino e de teste associado à run
    mlflow.log_input(train_dataset, context="train")
    mlflow.log_input(test_dataset, context="test")

    # Guardar a seed utilizado como parametro
    mlflow.log_param("seed", SEED)


    print(f"Treinando modelo e guardando todos os logs em MlFlow: {model_name}")
    start_time = time.time()
    
    # GridSearchCV para encontrar os melhores parâmetros
    grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
    grid_search.fit(X_train, y_train)

    end_time = time.time()
    elapsed_time = end_time - start_time  # tempo em segundos

    best_model = grid_search.best_estimator_

    # Guardar Log do modelo no MLflow
    mlflow.sklearn.log_model(
            sk_model=best_model,
            artifact_path=model_name,
            registered_model_name=model_name
    )

    # Guardar Log dos parâmetros e métricas do modelo
    mlflow.log_params(grid_search.best_params_)
    mlflow.log_metric("best_cv_score", grid_search.best_score_)
    mlflow.log_metric("training_time_sec", elapsed_time)  # tempo registado

    print(f"Modelo '{model_name}' guardado  MLflow.")
    mlflow.end_run()

Treinando modelo e guardando todos os logs em MlFlow: logistic_regression


c:\Users\PereiraRodrigues\miniconda3\envs\rumos_bank_lending\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
2025/04/08 23:35:08 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'logistic_regression' already exists. Creating a new version of this model...
2025/04/08 23:35:08 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: logistic_regre

Modelo 'logistic_regression' guardado  MLflow.
🏃 View run Ml Model Run - pipeline at: http://localhost:5000/#/experiments/395285683720049485/runs/d41fb87f52c342578bece7044fc6a56c
🧪 View experiment at: http://localhost:5000/#/experiments/395285683720049485
Treinando modelo e guardando todos os logs em MlFlow: knn


2025/04/08 23:35:12 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'knn' already exists. Creating a new version of this model...
2025/04/08 23:35:12 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: knn, version 6
Created version '6' of model 'knn'.


Modelo 'knn' guardado  MLflow.
🏃 View run Ml Model Run - pipeline at: http://localhost:5000/#/experiments/395285683720049485/runs/ce698dff59f4400880753351b60a24a9
🧪 View experiment at: http://localhost:5000/#/experiments/395285683720049485
Treinando modelo e guardando todos os logs em MlFlow: decision_tree


...

Modelo 'decision_tree' guardado  MLflow.
🏃 View run Ml Model Run - pipeline at: http://localhost:5000/#/experiments/395285683720049485/runs/c6040520a8d34802851e0198dc8c270e
🧪 View experiment at: http://localhost:5000/#/experiments/395285683720049485
Treinando modelo e guardando todos os logs em MlFlow: random_forest


2025/04/08 23:35:21 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'random_forest'.
2025/04/08 23:35:21 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: random_forest, version 1
Created version '1' of model 'random_forest'.


Modelo 'random_forest' guardado  MLflow.
🏃 View run Ml Model Run - pipeline at: http://localhost:5000/#/experiments/395285683720049485/runs/7b0746a28ce64f8ea38ea4789c69e12f
🧪 View experiment at: http://localhost:5000/#/experiments/395285683720049485
Treinando modelo e guardando todos os logs em MlFlow: mlp


2025/04/08 23:35:25 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'mlp'.
2025/04/08 23:35:25 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: mlp, version 1
Created version '1' of model 'mlp'.


Modelo 'mlp' guardado  MLflow.
🏃 View run Ml Model Run - pipeline at: http://localhost:5000/#/experiments/395285683720049485/runs/27338d50bbe54becac4fe0bbeb535052
🧪 View experiment at: http://localhost:5000/#/experiments/395285683720049485


In [289]:
#mlflow.end_run()